In [55]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from matplotlib import pyplot as plt
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools
import plotly.express as px 
from scipy import stats
from copy import deepcopy
from sklearn.model_selection import GroupShuffleSplit
from sklearn.datasets import fetch_openml
warnings.filterwarnings('ignore')

from Functions import dataprep as prep
from Functions import metrics
from sklearn.datasets import fetch_openml
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
train = pd.read_pickle('./data/common_dataprep/train.pickle')
test = pd.read_pickle('./data/common_dataprep/test.pickle')

In [42]:
# Expert input from Isabella
def build_predict(train, test, formula):
    glm1 = smf.glm(formula=formula, data=train, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train['Exposure'])).fit()
    preds_train = glm1.predict(train, offset=np.log(train['Exposure']))
    preds_test = glm1.predict(test, offset=np.log(test['Exposure']))
    
    return preds_train, preds_test

train['EI_Density'], test['EI_Density'] = build_predict(train, test, 'ClaimNb ~ Density') # GLM1, do not adjust

train['EI_DrivAge'], test['EI_DrivAge'] = build_predict(train, test, 'ClaimNb ~ DrivAge + I(DrivAge**2) + I(DrivAge**3) + I(DrivAge**4) + I(DrivAge**5)') # GLM5, do not adjust

train['EI_BonusMalus1'], test['EI_BonusMalus1'] = build_predict(train, test, 'ClaimNb ~ BonusMalus + I(BonusMalus**2)')

bm_below_100 = train.loc[train['BonusMalus'] <= 100, 'BonusMalus'].mean()
bm_above_100 = train.loc[train['BonusMalus'] > 100, 'BonusMalus'].mean()
train.loc[train['BonusMalus'] <= 100, 'EI_BonusMalus2'] = bm_below_100
test.loc[test['BonusMalus'] <= 100, 'EI_BonusMalus2'] = bm_below_100
train.loc[train['BonusMalus'] > 100, 'EI_BonusMalus2'] = bm_above_100
test.loc[test['BonusMalus'] > 100, 'EI_BonusMalus2'] = bm_above_100

train['EI_VehAge'], test['EI_VehAge'] = build_predict(train, test, 'ClaimNb ~ VehAge + I(VehAge**2) + I(VehAge**3)')

train.loc[train['VehAge'] <= 5, 'EI_VehAge'] = 0.05
test.loc[test['VehAge'] <= 5, 'EI_VehAge'] = 0.05

In [43]:
# Expert Input from GDV

# Vehicle Power
for df in train, test:
    df['GDV_Area'] = df['Area'].copy()
    df['GDV_Area'] = df['GDV_Area'].replace('A', 38.5)
    df['GDV_Area'] = df['GDV_Area'].replace('B', 41.5)
    df['GDV_Area'] = df['GDV_Area'].replace('C', 43.5)
    df['GDV_Area'] = df['GDV_Area'].replace('D', 46.5)
    df['GDV_Area'] = df['GDV_Area'].replace('E', (49+55)/2)
    df['GDV_Area'] = df['GDV_Area'].replace('F', (58+65)/2)

    # Vehicle Age
    df.loc[df['VehAge'] < 3, 'GDV_VehAge'] = 40
    df.loc[df['VehAge'] == 3, 'GDV_VehAge'] = 44
    df.loc[df['VehAge'].between(4, 5), 'GDV_VehAge'] = 46
    df.loc[df['VehAge'].between(6, 7), 'GDV_VehAge'] = 49
    df.loc[df['VehAge'] == 8, 'GDV_VehAge'] = 51
    df.loc[df['VehAge'] == 9, 'GDV_VehAge'] = 53
    df.loc[df['VehAge'].between(10, 11), 'GDV_VehAge'] = 57
    df.loc[df['VehAge'] == 12, 'GDV_VehAge'] = 61
    df.loc[df['VehAge'].between(13, 15), 'GDV_VehAge'] = 66
    df.loc[df['VehAge'].between(16, 17), 'GDV_VehAge'] = 69
    df.loc[df['VehAge'].between(18, 22), 'GDV_VehAge'] = 69
    df.loc[df['VehAge'] >= 23, 'GDV_VehAge'] = 34

    # Driver Age
    df.loc[df['DrivAge'] <= 18, 'GDV_DrivAge'] = 97
    df.loc[df['DrivAge'] == 19, 'GDV_DrivAge'] = 84
    df.loc[df['DrivAge'] == 20, 'GDV_DrivAge'] = 75
    df.loc[df['DrivAge'].between(21, 22), 'GDV_DrivAge'] = 67
    df.loc[df['DrivAge'].between(23, 24), 'GDV_DrivAge'] = 59
    df.loc[df['DrivAge'].between(25, 26), 'GDV_DrivAge'] = 67
    df.loc[df['DrivAge'].between(27, 41), 'GDV_DrivAge'] = 47
    df.loc[df['DrivAge'].between(42, 62), 'GDV_DrivAge'] = 37
    df.loc[df['DrivAge'].between(63, 67), 'GDV_DrivAge'] = 37
    df.loc[df['DrivAge'].between(68, 70), 'GDV_DrivAge'] = 40
    df.loc[df['DrivAge'].between(71, 72), 'GDV_DrivAge'] = 44
    df.loc[df['DrivAge'].between(73, 74), 'GDV_DrivAge'] = 48
    df.loc[df['DrivAge'].between(75, 76), 'GDV_DrivAge'] = 54
    df.loc[df['DrivAge'].between(77, 78), 'GDV_DrivAge'] = 58
    df.loc[df['DrivAge'].between(79, 81), 'GDV_DrivAge'] = 63
    df.loc[df['DrivAge'] >= 82, 'GDV_DrivAge'] = 74

In [44]:
train.to_pickle('./data/common_dataprep/train_with_ei.pickle')
test.to_pickle('./data/common_dataprep/test_with_ei.pickle')